In [1]:
# ============================================================
# nx47_vesu_kernel_v2.5
# ============================================================
import ctypes
import ast
import glob
import importlib
import io
import json
import os
import struct
import time
import zipfile
from dataclasses import dataclass
from hashlib import sha512
from pathlib import Path
from typing import Dict, List, Optional, Sequence, Tuple

try:
    from PIL import Image, ImageSequence
except Exception:  # pragma: no cover
    Image = None
    ImageSequence = None


class FatalPipelineError(RuntimeError):
    """Raised when strict fail-fast invariants are violated."""


@dataclass(frozen=True)
class CompatibilityLayer:
    name: str
    required_capabilities: List[str]


@dataclass(frozen=True)
class LUMVolume:
    shape: Tuple[int, int, int]
    dtype: str
    payload_sha512: str


class LumVoraxBridge:
    """Optional bridge to native LUM/VORAX C/C++ libraries via ctypes.

    Bridge is optional by design: pipeline remains fully functional in pure Python
    when no native shared library is available in Kaggle runtime.
    """

    def __init__(self) -> None:
        self.lib = None
        self.loaded_path = None

        candidates = [
            os.environ.get("LUM_VORAX_LIB_PATH", ""),
            "/kaggle/working/liblumvorax.so",
            "/kaggle/working/liblumvorax_replit.so",
            "/kaggle/working/libvorax.so",
            "/kaggle/input/lum-vorax-dependencies/liblumvorax.so",
            "/kaggle/input/lum-vorax-dependencies/libvorax.so",
            "/kaggle/input/datasets/ndarray2000/nx47-dependencies/liblumvorax_replit.so",
        ]
        for candidate in candidates:
            if candidate and os.path.exists(candidate):
                self.lib = ctypes.CDLL(candidate)
                self.loaded_path = candidate
                break

    @property
    def enabled(self) -> bool:
        return self.lib is not None


class NX47_VESU_Production:
    """NX-47 VESU production pipeline (Kaggle-ready, fail-fast, forensic).

    Key guarantees:
    - No synthetic fragment fallback.
    - Real TIFF input ingestion (2D or 3D, normalized to 3D).
    - Progressive multi-threshold 3D accumulation with target density clamping.
    - Optional `.lum` canonical intermediate serialization.
    - Optional native LUM/VORAX bridge via ctypes (non-blocking fallback).
    """

    ROADMAP_STEPS = [
        "bootstrap",
        "compatibility_check",
        "data_validation",
        "feature_extraction",
        "inference",
        "forensic_export",
        "finalize",
    ]

    LUM_MAGIC = b"LUMV1\x00\x00\x00"

    def __init__(self, input_dir=None, output_dir=None):
        self.version = "NX-47 VESU PROD V141-REAL-PY"
        self.audit_log: List[Dict] = []
        self.start_time = time.time_ns()
        env_input = os.environ.get("NX47_INPUT_DIR")
        self.input_dir = input_dir or env_input or "/kaggle/input/vesuvius-challenge-surface-detection"
        self.effective_input_root = self.input_dir
        self.discovery_attempts: List[Dict[str, object]] = []
        env_output = os.environ.get("NX47_OUTPUT_DIR")
        self.output_dir = output_dir or env_output or "/kaggle/working"
        self.processed_pixels = 0
        self.ink_detected = 0
        self.fallback_disabled = True
        self.roadmap_path = os.path.join(self.output_dir, "v141_roadmap_realtime.json")
        self.execution_log_path = os.path.join(self.output_dir, "v141_execution_logs.json")
        self.metadata_path = os.path.join(self.output_dir, "v141_execution_metadata.json")
        self.submission_zip_path = os.path.join(self.output_dir, "submission.zip")
        self.submission_parquet_path = os.path.join(self.output_dir, "submission.parquet")
        self.lum_work_dir = os.path.join(self.output_dir, "lum_cache")
        self.bridge = LumVoraxBridge()
        self.binary_mode = os.environ.get("NX47_BINARY_MODE", "0_1").strip().lower()
        if self.binary_mode not in {"0_1", "0_255"}:
            raise FatalPipelineError(f"INVALID_BINARY_MODE: {self.binary_mode}")

        self.capability_registry = {
            "preprocess_invariants": self.spatial_harmonic_filtering_simd,
            "input_format_guard": self._validate_input_structure,
            "feature_signature_v2": self._extract_fragment_signature,
            "intermediate_schema_v2": self._build_result_entry,
            "audit_hash_chain": self.log_event,
            "integrity_checks": self._integrity_digest,
            "forensic_traceability": self._export_forensic,
            "merkle_ready_events": self._audit_merkle_root,
            "realtime_roadmap": self._update_roadmap,
            "strict_train_evidence_gate": self._strict_training_evidence_gate,
            "adaptive_thresholding": self.ink_resonance_detector_v47,
            "dynamic_neuron_telemetry": self._emit_neuron_telemetry,
            "lum_encode_decode": self._roundtrip_lum,
        }

        self.compatibility_layers = [
            CompatibilityLayer("NX-1..NX-10", ["preprocess_invariants", "input_format_guard"]),
            CompatibilityLayer("NX-11..NX-20", ["feature_signature_v2", "intermediate_schema_v2"]),
            CompatibilityLayer("NX-21..NX-35", ["audit_hash_chain", "integrity_checks"]),
            CompatibilityLayer(
                "NX-36..NX-47",
                ["forensic_traceability", "merkle_ready_events", "realtime_roadmap", "dynamic_neuron_telemetry"],
            ),
            CompatibilityLayer(
                "NX-47 v115..v140",
                ["strict_train_evidence_gate", "adaptive_thresholding", "realtime_roadmap", "lum_encode_decode"],
            ),
        ]

        print(f"[{self.version}] System Initialized. Real TIFF processing + `.lum` roundtrip + fail-fast active.")

    @staticmethod
    def _is_pkg_available(package_name: str) -> bool:
        return importlib.util.find_spec(package_name) is not None

    @staticmethod
    def _matching_wheels(package_name: str, wheel_roots: Sequence[Path]) -> List[Path]:
        matches: List[Path] = []
        for root in wheel_roots:
            if not root.exists():
                continue
            matches.extend(sorted(root.glob(f"{package_name}-*.whl")))
        return matches

    def _run_pip_install(self, args: List[str], package_name: str) -> None:
        import subprocess

        timeout_s = int(os.environ.get("NX47_PIP_TIMEOUT_S", "180"))
        print(f"[{self.version}] PIP_INSTALL_START package={package_name} cmd={' ' .join(args)} timeout={timeout_s}s", flush=True)
        completed = subprocess.run(args, check=False, timeout=timeout_s)
        if completed.returncode != 0:
            raise subprocess.CalledProcessError(completed.returncode, args)
        print(f"[{self.version}] PIP_INSTALL_DONE package={package_name}", flush=True)

    def install_offline(self, package_name: str, required: bool = True) -> bool:
        import subprocess
        import sys

        if self._is_pkg_available(package_name):
            return True

        exact_wheel_dir = Path("/kaggle/input/datasets/ndarray2000/nx47-dependencies")
        fallback_wheel_dir = Path("/kaggle/input/nx47-dependencies")
        lum_wheel_dir = Path("/kaggle/input/lum-vorax-dependencies")
        lum_wheel_dir_alt = Path("/kaggle/input/lumvorax-dependencies")

        exact_wheels = {
            "imagecodecs": exact_wheel_dir / "imagecodecs-2026.1.14-cp311-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl",
            "numpy": exact_wheel_dir / "numpy-2.4.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl",
            "tifffile": exact_wheel_dir / "tifffile-2026.1.28-py3-none-any.whl",
        }

        if package_name in exact_wheels and exact_wheels[package_name].exists():
            try:
                self._run_pip_install([sys.executable, "-m", "pip", "install", "--disable-pip-version-check", "--no-index", str(exact_wheels[package_name])], package_name)
                if self._is_pkg_available(package_name):
                    return True
            except subprocess.CalledProcessError:
                pass

        wheel_roots = [exact_wheel_dir, fallback_wheel_dir, lum_wheel_dir, lum_wheel_dir_alt]

        for root in wheel_roots:
            if root.exists():
                try:
                    self._run_pip_install(
                        [sys.executable, "-m", "pip", "install", "--disable-pip-version-check", "--no-index", f"--find-links={root}", package_name],
                        package_name,
                    )
                    if self._is_pkg_available(package_name):
                        return True
                except Exception:
                    continue

        if required:
            raise FatalPipelineError(
                f"OFFLINE_DEPENDENCY_MISSING: {package_name} not found in known wheel directories."
            )
        raise FatalPipelineError(
            f"OPTIONAL_DEPENDENCY_POLICY_DISABLED: {package_name} missing and optional mode is forbidden."
        )

    def bootstrap_dependencies_fail_fast(self) -> None:
        # Optional bypass for local/Replit root validation preflight.
        if os.environ.get("NX47_SKIP_OFFLINE_BOOTSTRAP", "0").strip().lower() in {"1", "true", "yes"}:
            self.log_event("BOOTSTRAP_SKIPPED", {"reason": "NX47_SKIP_OFFLINE_BOOTSTRAP"})
            return

        # Required runtime packages
        for pkg in ("numpy", "pandas", "tifffile", "pyarrow"):
            self.install_offline(pkg, required=True)

        # Strict dependency policy: imagecodecs is mandatory and must fail-fast if missing.
        self.install_offline("imagecodecs", required=True)


    def _emit_dependency_resolution_report(self) -> None:
        wheel_roots = [
            "/kaggle/input/datasets/ndarray2000/nx47-dependencies",
            "/kaggle/input/nx47-dependencies",
            "/kaggle/input/lum-vorax-dependencies",
            "/kaggle/input/lumvorax-dependencies",
        ]
        report = {
            "wheel_roots": [{"path": p, "exists": os.path.exists(p)} for p in wheel_roots],
            "packages": {pkg: self._is_pkg_available(pkg) for pkg in ("numpy", "pandas", "tifffile", "pyarrow", "imagecodecs")},
        }
        self.log_event("DEPENDENCY_RESOLUTION_REPORT", report)

    def _emit_native_bridge_status(self) -> None:
        self.log_event(
            "NATIVE_BRIDGE_STATUS",
            {
                "enabled": bool(self.bridge.enabled),
                "path": self.bridge.loaded_path,
                "lum_magic": self.LUM_MAGIC.decode("latin1", errors="ignore"),
            },
        )

    def _memory_snapshot(self, stage: str) -> None:
        try:
            import resource
            rss_kb = int(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
        except Exception:
            rss_kb = -1
        self.log_event("MEMORY_SNAPSHOT", {"stage": stage, "rss_kb": rss_kb})

    @staticmethod
    def validate_source_indentation(source_path: Optional[str] = None) -> None:
        """Fail fast if source has indentation corruption (tabs/mixed blocks/syntax).

        Notebook-safe behavior:
        - in Kaggle/Papermill notebooks, ``__file__`` may be undefined;
        - we attempt multiple path candidates before deciding to skip the check.
        """
        import sys

        candidates: List[Path] = []
        if source_path:
            candidates.append(Path(source_path))
        else:
            g_file = globals().get("__file__")
            if isinstance(g_file, str) and g_file.strip():
                candidates.append(Path(g_file))
            argv0 = sys.argv[0] if sys.argv else ""
            if isinstance(argv0, str) and argv0.endswith(".py"):
                candidates.append(Path(argv0))
            candidates.append(Path.cwd() / "nx47_vesu_kernel_v2.4.py")

        target_path = next((c for c in candidates if c.exists() and c.is_file()), None)
        if target_path is None:
            # Notebook cell context without source file on disk.
            # We deliberately skip this guard instead of crashing the run.
            return

        content = target_path.read_text(encoding="utf-8")
        if "\t" in content:
            raise FatalPipelineError(f"SOURCE_INDENTATION_INVALID: tab characters found in {target_path}")
        try:
            ast.parse(content)
        except IndentationError as e:
            raise FatalPipelineError(f"SOURCE_INDENTATION_INVALID: {e}") from e
        except SyntaxError as e:
            raise FatalPipelineError(f"SOURCE_SYNTAX_INVALID: {e}") from e


    def log_event(self, event_type, details, severity="INFO"):
        ts = time.time_ns()
        previous_signature = self.audit_log[-1]["signature"] if self.audit_log else "GENESIS"
        log_entry = {
            "timestamp_ns": ts,
            "event": event_type,
            "severity": severity,
            "details": details,
            "previous_signature": previous_signature,
            "signature": sha512(f"{ts}{event_type}{details}{previous_signature}".encode()).hexdigest(),
        }
        self.audit_log.append(log_entry)

    def _update_roadmap(self, current_step, status="in_progress"):
        if current_step not in self.ROADMAP_STEPS:
            raise FatalPipelineError(f"Unknown roadmap step: {current_step}")
        current_idx = self.ROADMAP_STEPS.index(current_step)
        milestones = []
        for idx, step in enumerate(self.ROADMAP_STEPS):
            if idx < current_idx or (idx == current_idx and status == "done"):
                step_status = "done"
            elif idx == current_idx:
                step_status = "in_progress"
            else:
                step_status = "pending"
            milestones.append({"step": step, "status": step_status})
        roadmap = {
            "version": self.version,
            "timestamp_ns": time.time_ns(),
            "current_step": current_step,
            "status": status,
            "overall_progress_percent": round(
                ((current_idx + (1 if status == "done" else 0)) / len(self.ROADMAP_STEPS)) * 100, 2
            ),
            "milestones": milestones,
        }
        os.makedirs(self.output_dir, exist_ok=True)
        with open(self.roadmap_path, "w", encoding="utf-8") as f:
            json.dump(roadmap, f, indent=2)

    def _candidate_roots(self) -> List[Path]:
        configured = Path(self.input_dir)
        candidates = [
            configured,
            Path("/kaggle/input/vesuvius-challenge-surface-detection"),
            Path("/kaggle/input/competitions/vesuvius-challenge-surface-detection"),
            Path("/kaggle/input/vesuvius-challenge-ink-detection"),
            Path("/kaggle/input/competitions/vesuvius-challenge-ink-detection"),
        ]
        uniq: List[Path] = []
        seen = set()
        for c in candidates:
            k = str(c)
            if k not in seen:
                uniq.append(c)
                seen.add(k)
        return uniq

    def _resolve_input_root(self) -> Path:
        indicators = ("test", "test_scrolls", "test_fragments")
        for root in self._candidate_roots():
            has_indicator = any((root / name).exists() for name in indicators) or any(root.glob("**/test"))
            has_tiff = any(root.glob("**/*.tif")) or any(root.glob("**/*.tiff"))
            self.discovery_attempts.append(
                {"root": str(root), "exists": root.exists(), "has_indicator": bool(has_indicator), "has_tiff": bool(has_tiff)}
            )
            if root.exists() and (has_indicator or has_tiff):
                return root
        raise FatalPipelineError(
            f"INPUT_STRUCTURE_INVALID: no usable dataset root found. attempts={self.discovery_attempts}"
        )

    def _validate_input_structure(self):
        resolved = self._resolve_input_root()
        self.effective_input_root = str(resolved)

    def _collect_test_fragments(self) -> List[str]:
        root = Path(self.effective_input_root)
        candidates = []
        patterns = [
            str(root / "test" / "*.tif"),
            str(root / "test" / "*.tiff"),
            str(root / "test" / "**" / "*.tif"),
            str(root / "test" / "**" / "*.tiff"),
            str(root / "test_scrolls" / "**" / "*.tif"),
            str(root / "test_scrolls" / "**" / "*.tiff"),
            str(root / "test_fragments" / "**" / "*.tif"),
            str(root / "test_fragments" / "**" / "*.tiff"),
            str(root / "**" / "test" / "*.tif"),
            str(root / "**" / "test" / "*.tiff"),
            str(root / "**" / "surface_volume" / "*.tif"),
            str(root / "**" / "surface_volume" / "*.tiff"),
        ]
        for pattern in patterns:
            candidates.extend(glob.glob(pattern, recursive=True))

        uniq = sorted({str(Path(p)) for p in candidates})
        if not uniq:
            # last-resort recursive tif scan under root
            uniq = sorted({str(p) for p in root.rglob("*.tif")}) + sorted({str(p) for p in root.rglob("*.tiff")})
            uniq = sorted(set(uniq))
        if not uniq:
            raise FatalPipelineError(f"NO_TEST_FRAGMENTS_FOUND in {root}")
        return uniq

    def _validate_compatibility_chain(self):
        for layer in self.compatibility_layers:
            missing = [cap for cap in layer.required_capabilities if cap not in self.capability_registry]
            if missing:
                raise FatalPipelineError(f"COMPATIBILITY_BROKEN in {layer.name}: missing {missing}")
            self.log_event("COMPATIBILITY_LAYER_OK", {"layer": layer.name, "caps": layer.required_capabilities})

    def _strict_training_evidence_gate(self):
        expected = {
            "supervised_train": False,
            "val_f1_mean_supervised": None,
            "val_iou_mean_supervised": None,
            "native_bridge_enabled": self.bridge.enabled,
            "native_bridge_path": self.bridge.loaded_path,
        }
        self.log_event("STRICT_TRAINING_GATE", expected)

    @staticmethod
    def _normalize_volume_shape(volume: "np.ndarray") -> "np.ndarray":
        import numpy as np

        arr = np.asarray(volume)
        if arr.ndim == 2:
            return arr[np.newaxis, :, :]
        if arr.ndim == 3:
            return arr
        raise FatalPipelineError(f"UNSUPPORTED_VOLUME_DIM: expected 2D/3D, got {arr.ndim}D")

    def _read_tiff_volume(self, path: str) -> "np.ndarray":
        import numpy as np
        import tifffile

        try:
            arr = tifffile.imread(path)
        except ValueError as exc:
            if "requires the 'imagecodecs' package" in str(exc):
                raise FatalPipelineError(
                    f"DEPENDENCY_STRICT_FAILFAST: imagecodecs is required to decode TIFF LZW ({path})"
                ) from exc
            raise

        arr = self._normalize_volume_shape(arr)
        if arr.shape[1] <= 0 or arr.shape[2] <= 0:
            raise FatalPipelineError(f"INVALID_TIFF_SHAPE: {path} -> {arr.shape}")
        return arr.astype("float32", copy=False)

    def _lum_encode(self, volume: "np.ndarray") -> bytes:
        import numpy as np

        arr = self._normalize_volume_shape(volume)
        payload = np.ascontiguousarray(arr.astype(np.float32)).tobytes()
        digest = sha512(payload).digest()
        z, h, w = arr.shape
        header = struct.pack("<8sIII16s", self.LUM_MAGIC, z, h, w, digest[:16])
        return header + payload

    def _lum_decode(self, blob: bytes) -> "np.ndarray":
        import numpy as np

        header_size = struct.calcsize("<8sIII16s")
        if len(blob) < header_size:
            raise FatalPipelineError("LUM_DECODE_ERROR: blob too small")
        magic, z, h, w, digest16 = struct.unpack("<8sIII16s", blob[:header_size])
        if magic != self.LUM_MAGIC:
            raise FatalPipelineError("LUM_DECODE_ERROR: bad magic")
        payload = blob[header_size:]
        expected_bytes = int(z) * int(h) * int(w) * 4
        if len(payload) != expected_bytes:
            raise FatalPipelineError("LUM_DECODE_ERROR: payload size mismatch")
        if sha512(payload).digest()[:16] != digest16:
            raise FatalPipelineError("LUM_DECODE_ERROR: checksum mismatch")
        arr = np.frombuffer(payload, dtype=np.float32).reshape((z, h, w))
        return arr

    def _roundtrip_lum(self, volume: "np.ndarray") -> LUMVolume:
        blob = self._lum_encode(volume)
        decoded = self._lum_decode(blob)
        payload_sha = sha512(decoded.tobytes()).hexdigest()
        return LUMVolume(shape=tuple(decoded.shape), dtype=str(decoded.dtype), payload_sha512=payload_sha)

    def spatial_harmonic_filtering_simd(self, volume):
        import numpy as np

        vol = self._normalize_volume_shape(volume)
        filtered_slices = []
        for slice_data in vol:
            fft_data = np.fft.fft2(slice_data)
            mask = np.ones_like(slice_data, dtype=np.float32)
            rows, cols = slice_data.shape
            mask[rows // 4 : 3 * rows // 4, cols // 4 : 3 * cols // 4] = 0.5
            filtered_slices.append(np.abs(np.fft.ifft2(fft_data * mask)))
        return np.stack(filtered_slices, axis=0)

    @staticmethod
    def _clamp_density(mask_3d: "np.ndarray", density_target: float) -> "np.ndarray":
        import numpy as np

        total = mask_3d.size
        if total == 0:
            return mask_3d
        target_ones = int(total * density_target)
        if target_ones <= 0:
            return np.zeros_like(mask_3d, dtype=np.uint8)
        if target_ones >= total:
            return np.ones_like(mask_3d, dtype=np.uint8)

        flat = mask_3d.reshape(-1).astype(np.float32)
        if np.max(flat) <= 1.0 and np.min(flat) >= 0.0:
            scores = flat
        else:
            mn = float(np.min(flat))
            mx = float(np.max(flat))
            scores = (flat - mn) / (mx - mn + 1e-8)

        idx = np.argpartition(scores, -target_ones)[-target_ones:]
        out = np.zeros_like(scores, dtype=np.uint8)
        out[idx] = 1
        return out.reshape(mask_3d.shape)

    def ink_resonance_detector_v47(self, filtered_data):
        import numpy as np

        vol = self._normalize_volume_shape(filtered_data)
        mean = float(np.mean(vol))
        std = float(np.std(vol))
        thresholds = [mean + 0.8 * std, mean + 1.2 * std, mean + 1.6 * std]

        layer_low = (vol > thresholds[0]).astype(np.uint8)
        layer_mid = (vol > thresholds[1]).astype(np.uint8)
        layer_high = (vol > thresholds[2]).astype(np.uint8)

        # progressive accumulation with strictness weighting
        accum = (0.55 * layer_low + 0.30 * layer_mid + 0.15 * layer_high).astype(np.float32)
        binary = (accum >= 0.5).astype(np.uint8)

        # clamp to target density range to avoid extreme under/over segmentation
        density_now = float(np.mean(binary))
        target_density = min(max(density_now, 0.05), 0.12)
        return self._clamp_density(accum, target_density)

    def _extract_fragment_signature(self, fragment_id):
        return sha512(f"{fragment_id}|NX47".encode()).hexdigest()[:24]

    def _integrity_digest(self, payload):
        encoded = json.dumps(payload, sort_keys=True, default=str).encode()
        return sha512(encoded).hexdigest()

    def _build_result_entry(self, frag_id, score, density, shape):
        return {
            "id": frag_id,
            "target": float(score),
            "density": float(density),
            "shape_z": int(shape[0]),
            "shape_h": int(shape[1]),
            "shape_w": int(shape[2]),
            "feature_signature": self._extract_fragment_signature(frag_id),
        }

    def _emit_neuron_telemetry(self, filtered_data):
        import numpy as np

        vol = self._normalize_volume_shape(filtered_data)
        total = int(vol.size)
        active = int(np.count_nonzero(vol > np.mean(vol)))
        mid = int(np.count_nonzero(vol > (np.mean(vol) + 0.5 * np.std(vol))))
        end = int(np.count_nonzero(vol > (np.mean(vol) + 1.0 * np.std(vol))))
        return {
            "active_neurons_start_total": total,
            "active_neurons_mid_total": mid,
            "active_neurons_end_total": end,
            "mutation_events": 0,
            "pruning_events": 1,
        }

    def _audit_merkle_root(self):
        leaf_hashes = [entry["signature"] for entry in self.audit_log]
        if not leaf_hashes:
            return ""
        current = leaf_hashes
        while len(current) > 1:
            if len(current) % 2 == 1:
                current.append(current[-1])
            current = [sha512(f"{current[i]}{current[i + 1]}".encode()).hexdigest() for i in range(0, len(current), 2)]
        return current[0]


    def _to_binary_domain(self, mask: "np.ndarray") -> "np.ndarray":
        import numpy as np

        bin01 = (np.asarray(mask) > 0).astype(np.uint8)
        if self.binary_mode == "0_255":
            return (bin01 * 255).astype(np.uint8)
        return bin01

    def _write_submission_zip(self, masks: Dict[str, "np.ndarray"]) -> None:
        import tifffile

        os.makedirs(self.output_dir, exist_ok=True)
        with zipfile.ZipFile(self.submission_zip_path, "w", compression=zipfile.ZIP_STORED) as zf:
            for frag_id, mask in masks.items():
                arr = self._normalize_volume_shape(self._to_binary_domain(mask)).astype("uint8")
                tif_buf = io.BytesIO()
                try:
                    tifffile.imwrite(tif_buf, arr, compression="lzw")
                except Exception:
                    # imagecodecs may be missing in some Kaggle runs.
                    tifffile.imwrite(tif_buf, arr, compression=None)
                zf.writestr(f"{frag_id}.tif", tif_buf.getvalue())

    def _export_forensic(self, stats):
        os.makedirs(self.output_dir, exist_ok=True)
        with open(self.execution_log_path, "w", encoding="utf-8") as f:
            json.dump(self.audit_log, f, indent=2)

        metadata = {
            "version": self.version,
            "elapsed_total_s": round((time.time_ns() - self.start_time) / 1e9, 6),
            "integrity_digest": self._integrity_digest(stats),
            "merkle_root": self._audit_merkle_root(),
            "fallback_disabled": self.fallback_disabled,
            "native_bridge_enabled": self.bridge.enabled,
            "native_bridge_path": self.bridge.loaded_path,
            "submission_zip": self.submission_zip_path,
            "submission_parquet": self.submission_parquet_path,
            "binary_mode": self.binary_mode,
            "effective_input_root": self.effective_input_root,
            "discovery_attempts": self.discovery_attempts,
        }
        with open(self.metadata_path, "w", encoding="utf-8") as f:
            json.dump(metadata, f, indent=2)

    def process_fragments(self):
        import pandas as pd

        self._update_roadmap("bootstrap", "in_progress")
        self.log_event("PIPELINE_START", "Beginning fragment processing")
        print(f"[{self.version}] STAGE bootstrap:start", flush=True)
        self.bootstrap_dependencies_fail_fast()
        self._emit_dependency_resolution_report()
        self._emit_native_bridge_status()
        self._memory_snapshot("post_bootstrap")
        os.makedirs(self.lum_work_dir, exist_ok=True)

        self._strict_training_evidence_gate()
        self._update_roadmap("bootstrap", "done")
        print(f"[{self.version}] STAGE bootstrap:done", flush=True)

        self._update_roadmap("compatibility_check", "in_progress")
        self._validate_compatibility_chain()
        self._update_roadmap("compatibility_check", "done")
        print(f"[{self.version}] STAGE compatibility_check:done", flush=True)

        self._update_roadmap("data_validation", "in_progress")
        self._validate_input_structure()
        test_fragments = self._collect_test_fragments()
        self.log_event("TEST_FRAGMENT_DISCOVERY", {"count": len(test_fragments), "effective_root": self.effective_input_root, "attempts": self.discovery_attempts})
        self._update_roadmap("data_validation", "done")
        print(f"[{self.version}] STAGE data_validation:done fragments={len(test_fragments)}", flush=True)

        self._update_roadmap("feature_extraction", "in_progress")
        results = []
        telemetry = {
            "active_neurons_start_total": 0,
            "active_neurons_mid_total": 0,
            "active_neurons_end_total": 0,
            "mutation_events": 0,
            "pruning_events": 0,
        }
        masks_for_zip: Dict[str, "np.ndarray"] = {}

        total_frags = len(test_fragments)
        for idx, frag in enumerate(test_fragments, start=1):
            frag_id = os.path.splitext(os.path.basename(frag))[0]
            self.log_event("FRAGMENT_PROCESSING", {"fragment": frag_id, "path": frag})
            if idx == 1 or idx % 5 == 0 or idx == total_frags:
                print(f"[{self.version}] FRAGMENT_PROGRESS {idx}/{total_frags} id={frag_id}", flush=True)

            volume = self._read_tiff_volume(frag)
            lum_info = self._roundtrip_lum(volume)
            self.log_event("LUM_ROUNDTRIP_OK", {"fragment": frag_id, "shape": lum_info.shape, "dtype": lum_info.dtype})

            filtered = self.spatial_harmonic_filtering_simd(volume)
            pred = self.ink_resonance_detector_v47(filtered)

            score = float(pred.mean())
            density = float(pred.mean())
            results.append(self._build_result_entry(frag_id, score, density, pred.shape))

            masks_for_zip[frag_id] = pred
            self.processed_pixels += int(filtered.size)
            self.ink_detected += int(pred.sum())
            t = self._emit_neuron_telemetry(filtered)
            for k in telemetry:
                telemetry[k] += int(t.get(k, 0))

        self._update_roadmap("feature_extraction", "done")
        self._memory_snapshot("post_feature_extraction")
        print(f"[{self.version}] STAGE feature_extraction:done", flush=True)

        self._update_roadmap("inference", "in_progress")
        submission_df = pd.DataFrame(results)
        if submission_df.empty:
            raise FatalPipelineError("NO_RESULTS_GENERATED")
        submission_df[["id", "target"]].to_parquet(self.submission_parquet_path)
        self._write_submission_zip(masks_for_zip)
        self.log_event("SUBMISSION_GENERATED", {"shape": submission_df.shape, "zip": self.submission_zip_path})
        self._update_roadmap("inference", "done")
        print(f"[{self.version}] STAGE inference:done", flush=True)

        self._update_roadmap("forensic_export", "in_progress")
        stats = {
            "files_processed": len(results),
            "pixels_processed": self.processed_pixels,
            "ink_detected": self.ink_detected,
            "mean_density": float(submission_df["density"].mean()),
            **telemetry,
            "files_autonomous_fallback": 0,
            "lum_bridge_enabled": self.bridge.enabled,
        }
        self._export_forensic(stats)
        self._update_roadmap("forensic_export", "done")
        self._memory_snapshot("post_forensic_export")
        print(f"[{self.version}] STAGE forensic_export:done", flush=True)

        self._update_roadmap("finalize", "done")
        print(f"[{self.version}] Execution Complete.")
        return stats


if __name__ == "__main__":
    NX47_VESU_Production.validate_source_indentation()
    node = NX47_VESU_Production()
    node.process_fragments()


[NX-47 VESU PROD V141-REAL-PY] System Initialized. Real TIFF processing + `.lum` roundtrip + fail-fast active.
[NX-47 VESU PROD V141-REAL-PY] STAGE bootstrap:start
[NX-47 VESU PROD V141-REAL-PY] PIP_INSTALL_START package=imagecodecs cmd=/usr/bin/python3 -m pip install --disable-pip-version-check --no-index /kaggle/input/datasets/ndarray2000/nx47-dependencies/imagecodecs-2026.1.14-cp311-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl timeout=180s
Processing /kaggle/input/datasets/ndarray2000/nx47-dependencies/imagecodecs-2026.1.14-cp311-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
[NX-47 VESU PROD V141-REAL-PY] PIP_INSTALL_DONE package=imagecodecs
[NX-47 VESU PROD V141-REAL-PY] STAGE bootstrap:done
[NX-47 VESU PROD V141-REAL-PY] STAGE compatibility_check:done
[NX-47 VESU PROD V141-REAL-PY] STAGE data_validation:done fragments=1613
[NX-47 VESU PROD V141-REAL-PY] FRAGMENT_PROGRESS 1/1613 id=1407735
[NX-47 VESU PROD V141-REAL-PY] FRAGMENT_PROGRESS 5/1613 id=2075542469
[NX-47 V